In [1]:
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras

tf.enable_eager_execution()

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13582797715877038074
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4945621811
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15868177172779264386
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [ ]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [ ]:
tf.test.gpu_device_name()

In [3]:
tf.random.set_random_seed(22)
np.random.seed(22)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [4]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train.astype(np.float32)/255., x_test.astype(np.float32)/255.

In [5]:
x_train, x_test = np.expand_dims(x_train, axis=3), np.expand_dims(x_test, axis=3)

In [6]:
db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(256)
db_test = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(256)

In [7]:
class ConvBNRelu(keras.Model):
    
    def __init__(self, ch, kernelsz=3, strides=1, padding='same'):
        super(ConvBNRelu, self).__init__()
        
        self.model = keras.models.Sequential([
            keras.layers.Conv2D(ch, kernelsz, strides=strides, padding=padding),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU()
        ])
        
    def call(self, x, training=None):
        
        x = self.model(x, training=training)
        
        return x
    
class InceptionBlk(keras.Model):
    
    def __init__(self, ch, strides=1):
        super(InceptionBlk, self).__init__()
        
        self.ch = ch
        self.strides = strides
        
        self.conv1 = ConvBNRelu(ch, strides=strides)
        self.conv2 = ConvBNRelu(ch, kernelsz=3, strides=strides)
        self.conv3_1 = ConvBNRelu(ch, kernelsz=3, strides=strides)
        self.conv3_2 = ConvBNRelu(ch, kernelsz=3, strides=1)
        
        self.pool = keras.layers.MaxPooling2D(3, strides=1, padding='same')
        self.pool_conv = ConvBNRelu(ch, strides=strides)
    
    def call(self, x, training=None):
        
        x1 = self.conv1(x, training=training)
        
        x2 = self.conv2(x, training=training)
        
        x3_1 = self.conv3_1(x, training=training)
        x3_2 = self.conv3_2(x3_1, training=training)
        
        x4 = self.pool(x)
        x4 = self.pool_conv(x4, training=training)
        
        x = tf.concat([x1, x2, x3_2, x4], axis=3)
        
        return x

In [8]:
class Inception(keras.Model):
    
    def __init__(self, num_layers, num_classes, init_ch=16, **kwargs):
        super(Inception, self).__init__(**kwargs)
        
        self.in_channels = init_ch
        self.out_channels = init_ch
        self.num_layers = num_layers
        self.init_ch = init_ch
        
        self.conv1 = ConvBNRelu(init_ch)
        
        self.blocks = keras.models.Sequential(name='dynamic-blocks')
        
        for block_id in range(num_layers):
            
            for layer_id in range(2):
                
                if layer_id == 0:
                    
                    block = InceptionBlk(self.out_channels, strides=2)
                    
                else:
                    block = InceptionBlk(self.out_channels, strides=1)
                    
                self.blocks.add(block)
            
            # enlarger out_channels per block    
            self.out_channels *= 2
            
        self.avg_pool = keras.layers.GlobalAveragePooling2D()
        self.fc = keras.layers.Dense(num_classes)
        
        
    def call(self, x, training=None):
        
        out = self.conv1(x, training=training)
        
        out = self.blocks(out, training=training)
        
        out = self.avg_pool(out)
        out = self.fc(out)
        
        return out   
    
        

In [9]:
batch_size = 32
epochs = 100
model = Inception(2, 10)
model.build(input_shape=(None, 28, 28, 1))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_bn_relu (ConvBNRelu)    multiple                  224       
_________________________________________________________________
dynamic-blocks (Sequential)  multiple                  292704    
_________________________________________________________________
global_average_pooling2d (Gl multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  1290      
Total params: 294,218
Trainable params: 293,226
Non-trainable params: 992
_________________________________________________________________


In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
criteon = keras.losses.CategoricalCrossentropy(from_logits=True)

acc_meter = keras.metrics.Accuracy()

for epoch in range(100):
    
    for step, (x, y) in enumerate(db_train):
        
        with tf.GradientTape() as tape:
            logits=model(x)
            loss=criteon(tf.one_hot(y, depth=10), logits)
            
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        if step % 10 == 0:
            print(epoch, step, 'loss:', loss.numpy())
            
        acc_meter.reset_states()
        for x, y in db_test:
            logits = model(x, training=False)
            pred = tf.argmax(logits, axis=1)
            acc_meter.update_state(y, pred)
            
        print(epoch, 'evaluation_acc:', acc_meter.result().numpy())

Instructions for updating:
Use tf.cast instead.
0 0 loss: 2.3067923
0 evaluation_acc: 0.101
0 evaluation_acc: 0.2116
0 evaluation_acc: 0.2668
0 evaluation_acc: 0.2661
0 evaluation_acc: 0.127
0 evaluation_acc: 0.1057
0 evaluation_acc: 0.2288
0 evaluation_acc: 0.2727
0 evaluation_acc: 0.3359
0 evaluation_acc: 0.2651
0 10 loss: 2.096843
0 evaluation_acc: 0.3241
0 evaluation_acc: 0.2838
0 evaluation_acc: 0.4292
0 evaluation_acc: 0.3161
0 evaluation_acc: 0.5057
0 evaluation_acc: 0.353
0 evaluation_acc: 0.3798
0 evaluation_acc: 0.3829
0 evaluation_acc: 0.4507
0 evaluation_acc: 0.4559
0 20 loss: 1.575044
0 evaluation_acc: 0.5273
0 evaluation_acc: 0.5856
0 evaluation_acc: 0.5214
0 evaluation_acc: 0.4537
0 evaluation_acc: 0.3371
0 evaluation_acc: 0.5629
0 evaluation_acc: 0.5271
0 evaluation_acc: 0.5461
0 evaluation_acc: 0.5191
0 evaluation_acc: 0.4959
0 30 loss: 1.3221254
0 evaluation_acc: 0.5709
0 evaluation_acc: 0.6111
0 evaluation_acc: 0.6307
0 evaluation_acc: 0.5961
0 evaluation_acc: 0.6275

1 evaluation_acc: 0.9422
1 evaluation_acc: 0.9339
1 evaluation_acc: 0.9384
1 evaluation_acc: 0.9553
1 60 loss: 0.23070258
1 evaluation_acc: 0.9645
1 evaluation_acc: 0.9596
1 evaluation_acc: 0.9459
1 evaluation_acc: 0.9401
1 evaluation_acc: 0.9408
1 evaluation_acc: 0.9535
1 evaluation_acc: 0.9609
1 evaluation_acc: 0.9611
1 evaluation_acc: 0.9555
1 evaluation_acc: 0.9531
1 70 loss: 0.11910495
1 evaluation_acc: 0.9481
1 evaluation_acc: 0.9528
1 evaluation_acc: 0.9594
1 evaluation_acc: 0.9618
1 evaluation_acc: 0.9627
1 evaluation_acc: 0.9652
1 evaluation_acc: 0.965
1 evaluation_acc: 0.9603
1 evaluation_acc: 0.9588
1 evaluation_acc: 0.9606
1 80 loss: 0.15355435
1 evaluation_acc: 0.9594
1 evaluation_acc: 0.9623
1 evaluation_acc: 0.9651
1 evaluation_acc: 0.9646
1 evaluation_acc: 0.9626
1 evaluation_acc: 0.9624
1 evaluation_acc: 0.9604
1 evaluation_acc: 0.952
1 evaluation_acc: 0.9589
1 evaluation_acc: 0.9612
1 90 loss: 0.135211
1 evaluation_acc: 0.9576
1 evaluation_acc: 0.9528
1 evaluation_acc

2 evaluation_acc: 0.9796
2 evaluation_acc: 0.9802
2 evaluation_acc: 0.9814
2 evaluation_acc: 0.9811
2 evaluation_acc: 0.9785
2 evaluation_acc: 0.9727
2 evaluation_acc: 0.9678
2 evaluation_acc: 0.9733
2 130 loss: 0.13720977
2 evaluation_acc: 0.9799
2 evaluation_acc: 0.9773
2 evaluation_acc: 0.9672
2 evaluation_acc: 0.9611
2 evaluation_acc: 0.9744
2 evaluation_acc: 0.982
2 evaluation_acc: 0.9735
2 evaluation_acc: 0.9593
2 evaluation_acc: 0.9663
2 evaluation_acc: 0.9776
2 140 loss: 0.075022824
2 evaluation_acc: 0.9804
2 evaluation_acc: 0.9771
2 evaluation_acc: 0.9706
2 evaluation_acc: 0.9642
2 evaluation_acc: 0.9702
2 evaluation_acc: 0.9776
2 evaluation_acc: 0.9776
2 evaluation_acc: 0.9717
2 evaluation_acc: 0.9641
2 evaluation_acc: 0.9689
2 150 loss: 0.134992
2 evaluation_acc: 0.9787
2 evaluation_acc: 0.982
2 evaluation_acc: 0.9782
2 evaluation_acc: 0.9741
2 evaluation_acc: 0.9738
2 evaluation_acc: 0.9753
2 evaluation_acc: 0.9745
2 evaluation_acc: 0.973
2 evaluation_acc: 0.9742
2 evaluati